## <b><font color='darkblue'>Preface</font></b>
([source](https://courses.dataschool.io/view/courses/build-ai-agents-with-python/3021202-chapter-3-building-a-translation-app)) Building a translation app

### <b><font color='darkgreen'>Section 1: Translating to one language</font></b>
([source](https://courses.dataschool.io/view/courses/build-ai-agents-with-python/3021202-chapter-3-building-a-translation-app/9966136-translating-to-one-language))

In [1]:
import chatbot

translate_to_japanese_chatbot = chatbot.get(
    'Translate any input message into Japanese.',
    by_message='Bye')

In [2]:
translate_to_japanese_chatbot()

User: Nice to meet you!


AI: はじめまして！ (Hajimemashite!)




User: bye


AI: さようなら (Sayounara)

Or, more casually:

*   じゃあね (Jaa ne)
*   またね (Mata ne)
*   バイバイ (Bai bai)




User: exit


AI: Bye!


### <b><font color='darkgreen'>Section 2: Translating to any language</font></b>
([source](https://courses.dataschool.io/view/courses/build-ai-agents-with-python/3021202-chapter-3-building-a-translation-app/9966557-translating-to-any-language)) In this tutorial, you will add additional fields `language` to the state to define complex behavior without relying on the message list. ([details](https://langchain-ai.github.io/langgraph/tutorials/get-started/5-customize-state/))

In [10]:
from langgraph.graph import StateGraph, MessagesState, START
from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import MemorySaver


class CustomState(MessagesState):
    language: str


model = chatbot.get_model()
prompt = ChatPromptTemplate(
    [("system", "Translate the input from English to {language}."),
     ("placeholder", "{messages}")])
chain = prompt | model

In [8]:
def call_model(state: CustomState, chain=chain):
    updated_messages = chain.invoke(state)
    return {"messages": updated_messages}

In [11]:
workflow = StateGraph(CustomState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")

memory = MemorySaver()
app = workflow.compile(memory)

In [12]:
def translatebot(language: str):
    config = {"configurable": {"thread_id": 999}}

    while True:
        user_input = input("User:")

        if user_input in ["exit", "quit", "bye"]:
            print("AI: See you later!")
            break
        
        else:
            print("AI: ", end="")
            for chunk, metadata in app.stream({"messages": user_input, "language": language}, config, stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")

In [13]:
translatebot('Chinese')

User: Hello


AI: 你好 (Nǐ hǎo)




User: Bye


AI: 再见 (Zài jiàn)




User: exit


AI: See you later!
